In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
from bokeh.plotting import output_notebook
output_notebook()

# Hovering

Bokeh has simple hovering to make it easy to drill down on your data:


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.sampledata.iris import flowers

colormap = {'setosa': 'red', 'versicolor': 'green', 'virginica': 'blue'}
colors = [colormap[x] for x in flowers['species']]

tools='hover,pan,box_zoom,save'

p = figure(title = "Iris Morphology", tools=tools)
p.xaxis.axis_label = 'Petal Length'
p.yaxis.axis_label = 'Petal Width'

p.circle(flowers["petal_length"], flowers["petal_width"],
         color=colors, fill_alpha=0.2, size=10)

output_notebook()
show(p)

By default, Bokeh uses the *index* of the input dataframe in its hover panel.

## Exercise

- Load the cars dataset
- Add a column, "car year" that contains both the car model and the year. 
- Set that column as the index
- then use the Hover tool as above to show car mpg vs weight.


# Custom hover tips

Rather than setting the index, Bokeh lets you customize the hover tooltip itself, but it's a bit more verbose. On the plus side, any valid html [can be a tooltip](https://github.com/jni/blob-explorer/blob/bd9fa676a2a23317e2ea84bdf48b19e71b9e75d4/picker.py#L120), which means that you can actually embed images in the tooltip. This is great for exploring image data. (Though we won't see this here.)

In [ ]:
from bokeh.models import HoverTool

# make sure color is its own column
flowers['colors'] = colors

tools='hover,pan,box_zoom,save'

p = figure(title = "Iris Morphology", tools=tools)
p.xaxis.axis_label = 'Petal Length'
p.yaxis.axis_label = 'Petal Width'

p.circle('petal_width', 'petal_length', source=flowers,
         color='colors', fill_alpha=0.2, size=10)

# grab the hover tool and change the tooltip
p.select_one(HoverTool).tooltips = [
    #("tooltip name", "@column name"),
    ("Species", "@species"),
    ("Sepal length", "@sepal_length"),
    ("Sepal width", "@sepal_width"),
    ("Petal length", "@petal_length"),
    ("Petal width", "@petal_width"),
]

show(p) 

# Linked brushing

Sometimes the identity of points in different points can be traced by *linked brushing*, selecting a subset of data between different plots. Bokeh does this transparently as long as different figures share the same data source:

In [ ]:
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

x = list(range(-20, 21))
y0 = [abs(xx) for xx in x]
y1 = [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "pan,box_select,lasso_select,reset,save"

# create a new plot and add a renderer
left = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
right.circle('x', 'y1', source=source)

p = gridplot([[left, right]])

show(p)

## Exercise

- Draw two scatterplots from the cars dataset, using linked brushing.
- Repeat the asteroids visualization using Bokeh hover (show the asteroid "moID" at least), and using linked brushing. You may or may not want to set the background etc.